## Learn Python Meta Programming

In [2]:
## Typing of any variable
class Meta1:
    pass

def func1():
    pass

a = 1
b = 1.2
c = "hello"
d = True
e = None
f = Meta1()

print(a, b, c, d, e, f, func1)
print(type(a), type(b), type(c), type(d), type(e), type(f), type(func1))

# So type() function return a typing class of variable (even int, float, str and bool in Python is a class)

1 1.2 hello True None <__main__.Meta1 object at 0x7f7b185741f0> <function func1 at 0x7f7b1855d820>
<class 'int'> <class 'float'> <class 'str'> <class 'bool'> <class 'NoneType'> <class '__main__.Meta1'> <class 'function'>


In [3]:
import inspect

print(inspect.isclass(a))
print(inspect.isclass(type(a)))
print(inspect.isclass(f))
print(inspect.isclass(type(f)))
print(inspect.isclass(inspect.isclass(type(f))))
print(inspect.isclass(type(inspect.isclass(type(f)))))
print(type(type(f)))

# Looks like everything in Python is a object of a class => best place for meta programming :)

False
True
False
True
False
True
<class 'type'>


In [4]:
# Some common namespaces of a class

# So let print more readable with this command
from pprint import pprint

class Meta2:
    ''' Okay, this is document about Meta2 class '''
    var2 = 42
    def __init__(self) -> None:
        print("Hello from Meta2")
    
    def echo(self):
        ''' this is echoooooo function '''
        print("Echooooo")
        print(self)

print(type(Meta2.__dict__)) 
pprint(Meta2.__dict__) # Namespace of class

echo_from_meta2 = Meta2.__dict__.get('echo') # Try to get echo func from class namespace
try:
    echo_from_meta2("KhanhIceTea") # Pass anything to param "self", works ! self is not variable for current object in class context
    echo_from_meta2() # Keep param "self" empty => exception 
except Exception as e:
    print(e)
pprint(echo_from_meta2.__dict__) # function doesn't have namespace like class

<class 'mappingproxy'>
mappingproxy({'__dict__': <attribute '__dict__' of 'Meta2' objects>,
              '__doc__': ' Okay, this is document about Meta2 class ',
              '__init__': <function Meta2.__init__ at 0x7f7b1855dca0>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Meta2' objects>,
              'echo': <function Meta2.echo at 0x7f7b1855dd30>,
              'var2': 42})
Echooooo
KhanhIceTea
echo() missing 1 required positional argument: 'self'
{}


In [5]:
## Ok, deep dive into type() function

# print type func signature
print(type.__doc__)

# interesting ! so let try call type with 3 args as docs say "type(name, bases, dict) -> a new type"
# Becareful, bases arg is tupple of base classes
CloneMeta3 = type("CloneMeta2", (Meta2,), {"var3": 4})

# IDE with warning this class is not defined, but is it
print(CloneMeta3) # new class is belongs to current module run the type() func
print(CloneMeta3.var2, CloneMeta3.var3) # new SubClass inherits base classes
clone1 = CloneMeta3()
clone1.echo()
print(type(CloneMeta3))


type(object_or_name, bases, dict)
type(object) -> the object's type
type(name, bases, dict) -> a new type
<class '__main__.CloneMeta2'>
42 4
Hello from Meta2
Echooooo
<class 'type'>


In [6]:
# OK, nice lets try convert this
# class A:
#     var1 = 1
# class B(A):
#     var2 = 2
# by using type() function

A = type("A", (), {"var1": 1})
B = type("B", (A,), {"var2": 2})

print(A)
print(A.var1)
print(B)
print(B.var1)
print(B.var2)

<class '__main__.A'>
1
<class '__main__.B'>
1
2


## Decorators

In [7]:
def echo1(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo1 is echoing the msg '{msg}'")

print(echo1)
echo1("Hello world!")

# Now we create a decorator function to wrap (like Proxy Pattern in OOP)
def decor1(func):
    return func

decor1_echo1 = decor1(echo1) # this simplest decorator function
decor1_echo1("He he ha ha") # The same result as echo1 func

print(decor1_echo1)
print(decor1_echo1 == echo1)
print("It's have the same pointer address b/c return same function")



<function echo1 at 0x7f7b1855d5e0>
echo1 is echoing the msg 'Hello world!'
echo1 is echoing the msg 'He he ha ha'
<function echo1 at 0x7f7b1855d5e0>
True
It's have the same pointer address b/c return same function


In [8]:
# Go to more complex decorator

def decor2(func):
    def wrap(*args, **kwargs):
        print("Ehhhhh hemmmmm, calling from decor2 wrap function")
        ret = func(*args, **kwargs) # store returned value
        print("Yay, finished !")
        return ret
    return wrap

@decor2
def echo2(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo2 is echoing the msg '{msg}'")

decor2_echo1 = decor2(echo1)

print("=== Try to deepdive into some common namespaces of decorated function decor1_echo1")
pprint(decor2_echo1.__name__)
pprint(decor2_echo1.__dict__)
pprint(decor2_echo1.__dir__)
pprint(decor2_echo1.__doc__)
pprint(decor2_echo1.__annotations__)
pprint(decor2_echo1.__call__)

print("=== Try to deepdive into some common namespaces of decorated function echo2")
pprint(echo2.__name__)
pprint(echo2.__dict__)
pprint(echo2.__dir__)
pprint(echo2.__doc__)
pprint(echo2.__annotations__)
pprint(echo2.__call__)
print("=> As you can see, all namespaces of echo1, echo2 wouldn't be copied to final functions")


=== Try to deepdive into some common namespaces of decorated function decor1_echo1
'wrap'
{}
<built-in method __dir__ of function object at 0x7f7b1ae533a0>
None
{}
<method-wrapper '__call__' of function object at 0x7f7b1ae533a0>
=== Try to deepdive into some common namespaces of decorated function echo2
'wrap'
{}
<built-in method __dir__ of function object at 0x7f7b1860b820>
None
{}
<method-wrapper '__call__' of function object at 0x7f7b1860b820>
=> As you can see, all namespaces of echo1, echo2 wouldn't be copied to final functions


In [9]:
# So we have to copy namespace we need in decorated function
def decor3(func):
    def wrap(*args, **kwargs):
        print("Ehhhhh hemmmmm, calling from decor2 wrap function")
        ret = func(*args, **kwargs) # store returned value
        print("Yay, finished !")
        return ret
    wrap.__name__ = func.__name__
    wrap.__doc__ = func.__doc__
    wrap.__annotations__ = func.__annotations__
    return wrap

@decor3
def echo3(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo3 is echoing the msg '{msg}'")

echo3("OK")
print("=== Try to deepdive into some common namespaces of decorated function echo3")
pprint(echo3.__name__)
pprint(echo3.__doc__)
pprint(echo3.__annotations__)
print("=== See ! echo3 has its original namespaces")

Ehhhhh hemmmmm, calling from decor2 wrap function
echo3 is echoing the msg 'OK'
Yay, finished !
=== Try to deepdive into some common namespaces of decorated function echo3
'echo3'
' This func will echo loud to input message '
{'msg': <class 'str'>}
=== See ! echo3 has its original namespaces


In [10]:
# Copy namespace by using built-in function called wrap in functools module
from functools import wraps

def decor4(func):
    ''' 
    all it do is copy func's namespaces to wrap's namespaces :
        (__name__, __qualname__, __module__, __doc__, __annotations__)
    & updates __dict__ elements
    & point wrap.__wrapped__ to original func
    '''
    @wraps(func) 
    def wrap(*args, **kwargs):
        print("Ehhhhh hemmmmm, calling from decor2 wrap function")
        ret = func(*args, **kwargs) # store returned value
        print("Yay, finished !")
        return ret
    return wrap

@decor4
def echo4(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo4 is echoing the msg '{msg}'")

echo3("OK")
print("=== Try to deepdive into some common namespaces of decorated function echo4")

pprint(echo4.__name__)
pprint(echo4.__qualname__)
pprint(echo4.__module__)
pprint(echo4.__doc__)
pprint(echo4.__annotations__)
pprint(echo4.__wrapped__) # original echo4
pprint(echo4) # new echo4
print("=== See ! echo3 has its original namespaces")

Ehhhhh hemmmmm, calling from decor2 wrap function
echo3 is echoing the msg 'OK'
Yay, finished !
=== Try to deepdive into some common namespaces of decorated function echo4
'echo4'
'echo4'
'__main__'
' This func will echo loud to input message '
{'msg': <class 'str'>}
<function echo4 at 0x7f7b18503160>
<function echo4 at 0x7f7b185031f0>
=== See ! echo3 has its original namespaces


In [12]:
# Okay, you can see `wraps` built-in function is used as decorator it self :lol: and it has additional parameters,
# let's try !
# Basically, it's just a function that return a decorator

def decor5(bye_msg):
    def inner_decor(func):
        @wraps(func) 
        def wrap(*args, **kwargs):
            print("Ehhhhh hemmmmm, calling from decor2 wrap function")
            ret = func(*args, **kwargs) # store returned value
            print("Yay, finished ! ", bye_msg) # use `bye_msg` parameter here (like a closure)
            return ret
        return wrap
    return inner_decor

@decor5("Bye bye buddy !")
def echo5(msg: str):
    ''' This func will echo loud to input message '''
    print(f"echo5 is echoing the msg '{msg}'")

echo5("Hi Python community !")

Ehhhhh hemmmmm, calling from decor2 wrap function
echo5 is echoing the msg 'Hi Python community !'
Yay, finished !  Bye bye buddy !
